In [ ]:
!pip install langchain langchain-community llama-cpp-python transformers torch sentence-transformers faiss-gpu openai

In [18]:
from langchain_community.chat_models import ChatOpenAI
import openai
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter


openai.api_key = "sk-mDDbQXk0nqPVUKjReeOIgiBoIyyjANlaXcERYJqyM6T3BlbkFJ5V6MdFRVmWcxIG2n0JyM1woTAUDps1dWqv6uYvnKsA"

transcript_file = "/content/325-transcript.txt"

# 1. Load the document
loader = TextLoader(transcript_file)
documents = loader.load()

# 2. Split texts
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
texts = text_splitter.split_documents(documents)

# 3. Create embeddings
embeddings = HuggingFaceEmbeddings()

# 4. Create FAISS vector store
db = FAISS.from_documents(texts, embeddings)

# 5. Set up retriever
retriever = db.as_retriever()

# 6. Set up OpenAI LLM
llm = ChatOpenAI(
    model_name="gpt-4o-mini",  # or "gpt-3.5-turbo" if you prefer
    openai_api_key=openai.api_key
)

# 7. Create a prompt template
template = """
You are an assistant that provides answers to questions based on
a given context.

Answer the question based on the context. If you can't answer the
question, reply "I don't know".

Be as concise as possible and go straight to the point.

Context: {context}

Question: {question}
"""
prompt = PromptTemplate.from_template(template)

# 8. Set up the output parser
parser = StrOutputParser()

# 9. Combine the retriever, prompt, model, and parser into a chain
chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | llm
    | parser
)

# 10. Query the system
query = "What is the main topic of the transcripts and who are the interviewer and the interviewee?"
result = chain.invoke({"question": query})
print(result)

The main topic of the transcripts revolves around biological systems, regeneration, unconventional cognition, and the implications of these topics for understanding life and intelligence. The interviewer is Lex Fridman, and the interviewee is Michael Levin, a biologist.
